In [ ]:
import pinocchio as pin
import numpy as np
import proxsuite
import example_robot_data
from pinocchio.visualize import MeshcatVisualizer

In [ ]:
# load robot model
z1 = example_robot_data.load("z1")

viz = MeshcatVisualizer(z1.model, z1.collision_model, z1.visual_model)
viz.initViewer(loadModel=True, open=True)
# Load the robot in the viewer.
viz.loadViewerModel()
viz.displayVisuals(True)

In [ ]:
model, data = z1.model, z1.model.createData()

In [ ]:
# function to compute FK for all frames/joints
def fk_all(model, data, q, v = None):
    if v is not None:
        pin.forwardKinematics(model, data, q, v) # FK and Forward Velocities
    else:
        pin.forwardKinematics(model, data, q) # FK
    pin.updateFramePlacements(model, data) # Update frames

In [ ]:
def simu_step(model, data, q, v, v_target, dt):
    fk_all(model, data, q, v)
    a_desired = (v_target - v) / dt
    tau = pin.rnea(model, data, q, v, a_desired)
    tau = np.maximum(-model.effortLimit, np.minimum(tau, model.effortLimit))
    a = pin.aba(model, data, q, v, tau)
    vn = np.maximum(-model.velocityLimit, np.minimum(v + a * dt, model.velocityLimit))
    return pin.integrate(model, q, vn * dt), vn

In [ ]:
# Sample a random configuration as the target
pin.seed(np.random.randint(low=0, high=999999999)) # give a random seed to pinocchio
qd = pin.randomConfiguration(model)
print('qd:', qd.T)

In [ ]:
# We want to compute IK for our tip ("link06")
frame_id = model.getFrameId("link06")

In [ ]:
# first of all, let's compute the transformation matrix
fk_all(model, data, qd)
T_wd = data.oMf[frame_id].copy()

print(T_wd)

In [ ]:
# Let's create our QP controller!

# Let's make a QP object
qp_dim = model.nv
qp_dim_eq = 0 # no equality constraint!
qp_dim_in = model.nv # joint limits. ProxSuite supports double limits: d_min <= Cx <= d_max!
qp = proxsuite.proxqp.dense.QP(qp_dim, qp_dim_eq, qp_dim_in)

qp_init = False
prev_error = None
sum_error = 0.

def qp_control(model, data, q_k, T_wd, step, Kp = 1., Kd = 0.01, Ki = 0.):
    global qp
    global qp_init
    global prev_error
    global sum_error
    # first we need to compute FK
    fk_all(model, data, q_k)
    # We now need to get our current transformation matrix
    T_wb = data.oMf[frame_id]
    # We know compute the error
    error = pin.log(T_wb.actInv(T_wd)).vector
    # print(round(sum(np.square(error)), 2))
    if not qp_init:
        prev_error = np.copy(error)
    sum_error += (error * step)
    prev_error = np.copy(error)

    Vb = Kp * error + Kd * (error - prev_error) / step + Ki * sum_error
    # Compute Jacobian
    J = pin.computeFrameJacobian(model, data, q_k, frame_id, pin.ReferenceFrame.LOCAL) # Jacobian in local frame
    # Let's compute the QP matrices
    Q = J.T @ J
    q = -J.T @ Vb
    C = np.eye(model.nv) * step
    d_min = model.lowerPositionLimit - q_k
    d_max = model.upperPositionLimit - q_k
    if not qp_init: # in first iteration we initialize the model
        qp.init(Q, q, None, None, C, d_min, d_max)
    else: # otherwise, we update the model
        qp.update(Q, q, None, None, C, d_min, d_max)
        qp_init = True
    # Let's solve the QP
    qp.solve()
    # We get back the results
    v = np.copy(qp.results.x)
    return v

In [ ]:
# Let's create a random target
nq = model.nq # dim of joints

q = z1.q0.copy()
v = np.zeros_like(q)
dt = 0.005

viz.display(q)

In [ ]:
%%html
<iframe src="http://jupyter.lar.upatras.gr:7000/static/" width="640" height="480"></iframe>

In [ ]:
K = round(20. / dt) + 1
for k in range(K):
    # Let's compute the commands
    u = qp_control(model, data, q, T_wd, dt)
    # We set the commands
    q, v = simu_step(model, data, q, v, u, dt)
    # Visualize
    viz.display(q)